<a href="https://colab.research.google.com/github/VellummyilumVinoth/FineTune_ALBERT_NLP/blob/main/FineTune_ALBERT_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers

! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [2]:
from transformers import AlbertTokenizer, AlbertForMaskedLM
import torch
import tensorflow as tf
import os
import numpy as np
import re

# Load the ALBERT tokenizer and model
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [3]:
with open('The Hunger Games.txt','r') as f:
    file = f.read().split(".")

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line

text_lst = []
len_lst = []
for line in file:
    if len(line.split(" ")) >=20:
        line = clean_text(line)
        text_lst.append(line)
        len_lst.append(len(line.split(" ")))


In [4]:
text_lst

[' Even though it was years ago, I think he still remembers how I tried to drown him in a bucket when Prim brought him home.',
 ' I pull on trousers, a shirt, tuck my long dark braid up into a cap, and grab my forage bag.',
 ' On the table, under a wooden bowl to protect it from hungry rats and cats alike, sits a perfect little goat cheese wrapped in basil leaves.',
 ' Our part of District , nicknamed the Seam, is usually crawling with coal miners heading out to the morning shift at this hour.',
 ' Men and women with hunched shoulders, swollen knuckles, many who have long since stopped trying to scrub the coal dust out of their broken nails, the lines of their sunken faces.',
 ' Separating the Meadow from the woods, in fact enclosing all of District , is a high chain link fence topped with barbed wire loops.',
 ' In theory, its supposed to be electrified twenty four hours a day as a deterrent to the predators that live in the woods packs of wild dogs, lone cougars, bears that used to t

In [5]:
train_size = int(len(text_lst) * 0.8)
train_size

1253

In [6]:
train_lst = text_lst[:train_size]
test_lst = text_lst[train_size:]

In [7]:
train_dataset = tokenizer(train_lst,max_length=100,truncation=True,padding='max_length',return_tensors='tf')

print(train_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [8]:
test_dataset = tokenizer(test_lst,max_length=100,truncation=True,padding='max_length',return_tensors='tf')

print(test_dataset.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [9]:
train_dataset['labels'] = train_dataset['input_ids']
train_dataset.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [10]:
test_dataset['labels'] = test_dataset['input_ids']
test_dataset.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [11]:
test = np.array(test_dataset.input_ids)
test_ids = torch.tensor(test)

test_ids = test_ids.to(torch.long)

test_ids[0]

tensor([    2,   101,   184,    31,   404,    31,   491,    44,   543,    20,
          308,    76,    24,    29, 12735,    32,    17,   184,    31,   404,
           14,  1742, 24131,   114,  2672,   301,    20,  6378, 13358,  6922,
         1257,    17,   184,    31,   143,   130,    30,    31,    57,    20,
          418,    71,    14,  6409,     9,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [12]:
inp_ids = []
lbs = []
idx = 0
for inp in train_dataset.input_ids.numpy():
    actual_tokens = list(set(range(100)) - 
                         set(np.where((inp == 101) | (inp == 102) 
                            | (inp == 0))[0].tolist()))
    # We need to select 15% random tokens from the given list
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens), 
                                     size=num_of_token_to_mask, 
                                     replace=False).tolist()
    # Now we have the indices where we need to mask the tokens
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
train = np.array(inp_ids)
train_ids = torch.tensor(train)
train_dataset['input_ids'] = train_ids

In [13]:
train_ids = train_ids.to(torch.long)

train_ids[0]

tensor([    2,   166,   362,    32,   103,   122,  1464,    15,    31,   277,
           24,   174,  1518,    18,   103,    31,   794,    20, 20380,    61,
          103,    21, 12433,    76,  4566,    79,   809,    61,   213,     9,
          103,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [21]:
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 100,                # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 1e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_ids,
    eval_dataset = test_ids,
    data_collator = data_collator,
)

trainer.train()

# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/spiece.model',
 './finetuned_albert/added_tokens.json')

In [25]:
#Fine tune ALBERT model with hunger games.txt file

import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)

# Define a sample masked statement
# masked_statement = "The quick brown [MASK] jumps over the lazy dog."
masked_statement = "Good morning [MASK], have a good day."
# masked_statement = "Family [MASK] line up around the perimeter, holding tightly to one anothers hands."
# masked_statement = "It was good hearty [MASK], filled with raisins and nuts."

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=True, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores
probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k = torch.topk(probs, k=5)

# Print the top 5 predictions and their probability scores
for i, idx in enumerate(top_k.indices):
    token = tokenizer.convert_ids_to_tokens([idx])[0]
    print(f"Prediction {i+1}: {token}, Probability: {top_k.values[i].item()}")


Prediction 1: ▁everyone, Probability: 0.12222923338413239
Prediction 2: ▁welcome, Probability: 0.0510295107960701
Prediction 3: ▁friends, Probability: 0.03291067108511925
Prediction 4: ▁everybody, Probability: 0.0323101207613945
Prediction 5: ▁tomorrow, Probability: 0.025291113182902336


In [26]:
# without fine tune the ALBERT model 

from transformers import pipeline

unmasker = pipeline('fill-mask', model='albert-base-v2')

# unmasker("The quick brown [MASK] jumps over the lazy dog.")
unmasker("Good morning [MASK], have a good day.")
# unmasker("Family [MASK] line up around the perimeter, holding tightly to one anothers hands.")
# unmasker("It was good hearty [MASK], filled with raisins and nuts.")

[{'score': 0.35843220353126526,
  'token': 29833,
  'token_str': 'evalle',
  'sequence': 'good morning evalle, have a good day.'},
 {'score': 0.029628073796629906,
  'token': 28153,
  'token_str': 'joyah',
  'sequence': 'good morning joyah, have a good day.'},
 {'score': 0.009840612299740314,
  'token': 14563,
  'token_str': 'folks',
  'sequence': 'good morning folks, have a good day.'},
 {'score': 0.009558583609759808,
  'token': 2776,
  'token_str': 'guys',
  'sequence': 'good morning guys, have a good day.'},
 {'score': 0.007196676917374134,
  'token': 2247,
  'token_str': 'please',
  'sequence': 'good morning please, have a good day.'}]